## 使用 GET 爬取 PTT 資料

In [2]:
import requests
url = 'https://www.ptt.cc/bbs/creditcard/index.html'
response = requests.get(url)
#print(response.text)

## 使用 POST 抓取高鐵時刻表

In [4]:
import requests
url = 'https://www.thsrc.com.tw/TimeTable/Search'
payload = {
  'SearchType': 'S',
  'Lang': 'TW',
  'StartStation': 'TaiPei',
  'EndStation': 'XinZhu',
  'OutWardSearchDate': '2023/04/15',
  'OutWardSearchTime': '18:00',
  'ReturnSearchDate': '2023/04/03',
  'ReturnSearchTime': '10:00',
  'DiscountType': ''
}
response = requests.post(url, data=payload)
#print(response.json())

## 練習題
請觀察以下頁面該是使用 GET or POST 抓取頁面資料?
- https://www.mobile01.com/topicdetail.php?f=180&t=6741959 

## 使用 BeautifulSoup 剖析頁面

### BeautifulSoup 範例

In [7]:
# BeautifulSoup 範例
from bs4 import BeautifulSoup 
html_sample = ''' 
<html> 
 <body> 
 <h1 id="title">Hello World</h1> 
 <a href="#" class="link">This is link1</a> 
 <a href="# link2" class="link">This is link2</a> 
 </body> 
 </html>'''

soup = BeautifulSoup(html_sample, 'lxml')
print(soup.text)



Hello World
This is link1
This is link2




### 取出h1 標籤的資料

In [8]:
# 取出h1 標籤的資料
soup = BeautifulSoup(html_sample, 'lxml') 
title = soup.select_one('h1') 
print(title)


<h1 id="title">Hello World</h1>


### 找出所有含a tag 的HTML 元素

In [9]:
# 找出所有含a tag 的HTML 元素
soup = BeautifulSoup(html_sample, 'lxml') 
alink = soup.select('a') 
print(alink)

[<a class="link" href="#">This is link1</a>, <a class="link" href="# link2">This is link2</a>]


### 取得含有特定ID的元素

In [10]:
# 取得含有特定ID的元素
alink = soup.select('#title') 
print(alink)


[<h1 id="title">Hello World</h1>]


### 取得含有特定class的元素

In [11]:

# 取得含有特定class的元素
soup = BeautifulSoup(html_sample, 'lxml') 
for link in soup.select('.link'): 
    print(link)



<a class="link" href="#">This is link1</a>
<a class="link" href="# link2">This is link2</a>


### 取得所有a tag 內的連結

In [12]:
# 取得所有a tag 內的連結
alinks = soup.select('a') 
for link in alinks: 
    print(link['href'])

#
# link2


## 資料蒐集實務

In [ ]:
#抓取列表原始碼
import requests
from bs4 import BeautifulSoup
res = requests.get('https://www.ptt.cc/bbs/creditcard/index.html')

#用迴圈遍歷含有 r-ent 的元素
soup = BeautifulSoup(res.text , 'lxml')
for rec in soup.select('.r-ent'):
    print(rec)


In [16]:
for rec in soup.select('.r-ent'):
 if rec.select_one('a'):
  title = rec.select_one('a').text
  link  = rec.select_one('a').get('href')
  author = rec.select_one('.author').text
  push  = rec.select_one('.nrec').text
  date  = rec.select_one('.date').text
  print(title, link, author, push, date)


[心得] 2023上半年用卡整理  4/10更新 /bbs/creditcard/M.1672850327.A.94E.html cokelon 爆  1/05
[心得] 2023Q2 信用卡回饋指定通路 /bbs/creditcard/M.1680335399.A.AA6.html SmallBeeWayn 13  4/01
[討論] 湊首刷/滿額/次數/門檻的儲值管道整理 /bbs/creditcard/M.1512475390.A.102.html kobe741107 93 12/05
[公告] 置底閒聊專區；宜：先詳閱板規,再發文/言 /bbs/creditcard/M.1672891679.A.07F.html tourism1003 19  1/05
[公告] 卡板板規 V9.1 / 置底檢舉專區 /bbs/creditcard/M.1615816709.A.BF3.html tourism1003 5  3/15


### 擷取內文

In [17]:
from bs4 import BeautifulSoup
import requests

res = requests.get('https://www.ptt.cc/bbs/creditcard/M.1680163258.A.82D.html')
soup = BeautifulSoup(res.text, 'lxml')

main_content = soup.select_one('#main-content')

# 移除 class 為 article-meta-tag, article-meta-value, f2, f6 的標籤
for tag in main_content.select('.article-meta-tag, .article-meta-value, .f6, .f2'):
  tag.decompose()

# 取得 -- 之前的所有文字
content = main_content.get_text('\n', strip=True)
content = content.split('--')[0].strip()
print(content)


中信101聯名卡新推出邀約制的尊榮鼎極VISA無限卡金屬卡

邀請條件是年消費300萬元或傳富家(年資產往來3000萬)

一般消費現金回饋1％無上限、海外消費現金回饋3%無上限

101觀景台門票與快速通關8折

停車和原本的無限卡一樣4小時

其他權益如餐廳保留位等也都跟原本的無限卡差不多

年費NT28,000元，本行臻、傳富家資格及101尊榮俱樂部會員資格免年費。
https://www.cardu.com.tw/news/detail.php?48533
https://i.imgur.com/YtuHoAl.jpg
上次華航鼎尊無限卡限量3000張發了三年還沒發完，這次101尊榮鼎極卡能發出幾張?


### 擷取評論

In [18]:
for push in soup.select('.push'):
 push_tag = push.select_one('.push-tag').text.strip()
 push_userid = push.select_one('.push-userid').text.strip()
 push_content = push.select_one('.push-content').text.strip()
 push_updatetime = push.select_one('.push-ipdatetime').text.strip()
 print(push_tag,push_userid, push_content, push_updatetime)


推 FMANT : 華航鼎尊連附卡都是金屬卡，有限量？呵呵 03/30 16:37
推 XXXXXXXCC : 這優惠有很好嗎？ 03/30 16:39
→ catuncle1 : 如果只是極富尊貴，但權益soso，還收高額年費，一般 03/30 17:04
→ catuncle1 : 卡友是不會買單，看那些常逛101有錢人會不會有興趣 03/30 17:05
推 jameswang278 : 看起來跟原本的鼎極卡沒啥太大差別，真正頂端客戶應 03/30 20:08
→ jameswang278 : 該也不差那一丁點的權益.... 03/30 20:08
→ timsky : 無感 不會辦 03/30 20:57
推 Jack131 : 發那張真的鼎尊卡還比較實在 03/30 22:59
推 cka : 看來根本沒人會為了這些權益繳28000啦，那是寫給免 04/01 00:21
→ cka : 年費那些人看的，說你看喔你賺到28000了 04/01 00:21
推 abyssa1 : 這是給理財客戶尊榮感的 發幾張不是重點 04/07 23:54
→ abyssa1 : 推這個比較像是中信打算分享給101客戶資料 04/07 23:55
→ abyssa1 : 手上拿這張就是30M身價起跳的潛在大戶 04/07 23:55


### 將內文抓取腳本整理成函數

In [20]:
def get_main_content(url):
 res = requests.get(url)
 soup = BeautifulSoup(res.text, 'lxml')

 main_content = soup.select_one('#main-content')

 for tag in main_content.select('.article-meta-tag, .article-meta-value, .f6, .f2'):
   tag.decompose()

 content = main_content.get_text('\n', strip=True)
 content = content.split('--')[0].strip()
 return content


### 將推文抓取腳本整理成函數

In [21]:
def get_push_content(url):
 res = requests.get(url)
 soup = BeautifulSoup(res.text, 'lxml')
 push_ary = []
 for push in soup.select('.push'):
  push_dic = {}
  push_dic['push_tag'] = push.select_one('.push-tag').text.strip()
  push_dic['push_userid'] = push.select_one('.push-userid').text.strip()
  push_dic['push_content'] = push.select_one('.push-content').text.strip()
  push_dic['push_updatetime'] = push.select_one('.push-ipdatetime').text.strip()
  push_ary.append(push_dic)
 return push_ary


## 完整抓取所有評論

### 將清單連結傳給評論抓取函數

In [26]:
domain = 'https://www.ptt.cc'
data_ary = []
for i in range(3692, 3650, -1):
 res = requests.get(f'https://www.ptt.cc/bbs/creditcard/index{i}.html')
 soup = BeautifulSoup(res.text, 'lxml')
 for rec in soup.select('.r-ent'):
  if rec.select_one('a'):
   data_dic = {}
   data_dic['title'] = rec.select_one('a').text
   data_dic['link']  = domain + rec.select_one('a').get('href')
   data_dic['author'] = rec.select_one('.author').text
   data_dic['push']  = rec.select_one('.nrec').text
   data_dic['date']  = rec.select_one('.date').text
   data_dic['content']= get_main_content(data_dic['link'])
   data_ary.append(data_dic)

### 將資料轉到Pandas 的DataFrame

In [27]:
import pandas
data_df = pandas.DataFrame(data_ary)
data_df.head()


,title,link,author,push,date,content
0,Re: [情報] 2023 凱基行動支付1.5%/4% 悠遊卡速食8%活動,https://www.ptt.cc/bbs/creditcard/M.1681312432...,thomson,9,4/12,放榜了\n凱基魔Buy+橘子支付 繳費有4%回饋\n凱基魔Buy+全盈PAY 繳費有 1~3...
1,[心得] 台新 GoGo&Flygo核卡,https://www.ptt.cc/bbs/creditcard/M.1681343426...,peter120,5,4/13,★職業類別：通訊產業\n\n★年資：1.5y\n\n★年齡：25\n\n★申請卡片：@GoG...
2,[閒聊] 花旗卡盜刷,https://www.ptt.cc/bbs/creditcard/M.1681358049...,t1013tom,13,4/13,平常都在板上潛水看大神們分享的攻略\n每次看到盜刷文都覺得不太乾我的事，因為我比較少用網拍買...
3,[心得] 兆豐新光核卡 渣打花旗婉拒 多家調額,https://www.ptt.cc/bbs/creditcard/M.1681373559...,yeuling9300,,4/13,職業類別：出版業\n年資：2y\n年齡：29\n申請卡片：兆豐宇宙明星BT21信用卡/新光O...
4,[心得] 富邦好市多鈦金相片卡核卡,https://www.ptt.cc/bbs/creditcard/M.1681374700...,gyqmo,7,4/13,★職業類別：教育業\n\n★年資：10\n\n★年齡：37\n\n★申請卡片：富邦好市多相片...


### 使用Pandas 匯出資料

In [28]:
data_df.to_excel('ptt_creditcard.xlsx')